# Trimodal preprocessing

In [1]:
import warnings
warnings.filterwarnings('ignore')

import scanpy as sc
import numpy as np
import pandas as pd
import anndata as ad
import muon

## Read the data

In [2]:
rna_multiome = sc.read('data/multiome/rna_multiome.h5ad')
rna_multiome

AnnData object with n_obs × n_vars = 69249 × 13431
    obs: 'GEX_pct_counts_mt', 'GEX_n_counts', 'GEX_n_genes', 'GEX_size_factors', 'GEX_phase', 'ATAC_nCount_peaks', 'ATAC_atac_fragments', 'ATAC_reads_in_peaks_frac', 'ATAC_blacklist_fraction', 'ATAC_nucleosome_signal', 'cell_type', 'batch', 'ATAC_pseudotime_order', 'GEX_pseudotime_order', 'Samplename', 'Site', 'DonorNumber', 'Modality', 'VendorLot', 'DonorID', 'DonorAge', 'DonorBMI', 'DonorBloodType', 'DonorRace', 'Ethnicity', 'DonorGender', 'QCMeds', 'DonorSmoker'
    var: 'feature_types', 'gene_id', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'highly_variable_nbatches', 'highly_variable_intersection'
    uns: 'ATAC_gene_activity_var_names', 'dataset_id', 'genome', 'hvg', 'log1p', 'organism'
    obsm: 'ATAC_gene_activity', 'ATAC_lsi_full', 'ATAC_lsi_red', 'ATAC_umap', 'GEX_X_pca', 'GEX_X_umap'
    layers: 'counts'

In [3]:
rna_cite = sc.read('data/cite/rna_cite.h5ad')
rna_cite

AnnData object with n_obs × n_vars = 90261 × 13953
    obs: 'GEX_n_genes_by_counts', 'GEX_pct_counts_mt', 'GEX_size_factors', 'GEX_phase', 'ADT_n_antibodies_by_counts', 'ADT_total_counts', 'ADT_iso_count', 'cell_type', 'batch', 'ADT_pseudotime_order', 'GEX_pseudotime_order', 'Samplename', 'Site', 'DonorNumber', 'Modality', 'VendorLot', 'DonorID', 'DonorAge', 'DonorBMI', 'DonorBloodType', 'DonorRace', 'Ethnicity', 'DonorGender', 'QCMeds', 'DonorSmoker', 'is_train'
    var: 'feature_types', 'gene_id', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'highly_variable_nbatches', 'highly_variable_intersection'
    uns: 'dataset_id', 'genome', 'hvg', 'log1p', 'organism'
    obsm: 'ADT_X_pca', 'ADT_X_umap', 'ADT_isotype_controls', 'GEX_X_pca', 'GEX_X_umap'
    layers: 'counts'

In [4]:
atac_hvf = sc.read('data/multiome/atac_hvf.h5ad')
atac_hvf

AnnData object with n_obs × n_vars = 69249 × 20000
    obs: 'GEX_pct_counts_mt', 'GEX_n_counts', 'GEX_n_genes', 'GEX_size_factors', 'GEX_phase', 'ATAC_nCount_peaks', 'ATAC_atac_fragments', 'ATAC_reads_in_peaks_frac', 'ATAC_blacklist_fraction', 'ATAC_nucleosome_signal', 'cell_type', 'batch', 'ATAC_pseudotime_order', 'GEX_pseudotime_order', 'Samplename', 'Site', 'DonorNumber', 'Modality', 'VendorLot', 'DonorID', 'DonorAge', 'DonorBMI', 'DonorBloodType', 'DonorRace', 'Ethnicity', 'DonorGender', 'QCMeds', 'DonorSmoker'
    var: 'feature_types', 'gene_id', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'highly_variable_nbatches', 'highly_variable_intersection'
    uns: 'ATAC_gene_activity_var_names', 'dataset_id', 'genome', 'hvg', 'log1p', 'organism'
    obsm: 'ATAC_gene_activity', 'ATAC_lsi_full', 'ATAC_lsi_red', 'ATAC_umap', 'GEX_X_pca', 'GEX_X_umap'
    layers: 'counts', 'log-norm', 'tf-idf'

In [5]:
adt = sc.read('data/cite/adt.h5ad')
adt

AnnData object with n_obs × n_vars = 90261 × 134
    obs: 'GEX_n_genes_by_counts', 'GEX_pct_counts_mt', 'GEX_size_factors', 'GEX_phase', 'ADT_n_antibodies_by_counts', 'ADT_total_counts', 'ADT_iso_count', 'cell_type', 'batch', 'ADT_pseudotime_order', 'GEX_pseudotime_order', 'Samplename', 'Site', 'DonorNumber', 'Modality', 'VendorLot', 'DonorID', 'DonorAge', 'DonorBMI', 'DonorBloodType', 'DonorRace', 'Ethnicity', 'DonorGender', 'QCMeds', 'DonorSmoker', 'is_train'
    var: 'feature_types', 'gene_id'
    uns: 'dataset_id', 'genome', 'organism'
    obsm: 'ADT_X_pca', 'ADT_X_umap', 'ADT_isotype_controls', 'GEX_X_pca', 'GEX_X_umap'
    layers: 'clr', 'counts'

## Add harmonized cell types

### Multiome

In [6]:
multiome_celltypes = pd.read_csv('data/trimodal/multiome_harmonized_celltypes.csv', index_col=0)
multiome_celltypes

,l1_cell_type,l2_cell_type
s1d1_multiome:TAGTTGTCACCCTCAC,B,Naive CD20+ B
s1d1_multiome:CTATGGCCATAACGGG,Myeloid,CD14+ Mono
s1d1_multiome:CCGCACACAGGTTAAA,T,CD8+ T activated
s1d1_multiome:TCATTTGGTAATGGAA,T,CD8+ T activated
s1d1_multiome:ACCACATAGGTGTCCA,Myeloid,CD16+ Mono
...,...,...
s4d9_multiome:AAACCGCGTTTGAGGC,T,CD8+ T naive
s4d9_multiome:TGACTTAAGTTCCCGT,Other Lympoid,Early Lymphoid
s4d9_multiome:GCTGTACCACCGTTCC,T,CD8+ T activated
s4d9_multiome:ACACTTGCAACTAGAA,Myeloid,cDC2


In [7]:
rna_multiome.obs_names

Index(['TAGTTGTCACCCTCAC-1-s1d1', 'CTATGGCCATAACGGG-1-s1d1',
       'CCGCACACAGGTTAAA-1-s1d1', 'TCATTTGGTAATGGAA-1-s1d1',
       'ACCACATAGGTGTCCA-1-s1d1', 'TGGATTGGTTTGCGAA-1-s1d1',
       'GTGAGCGAGTAAAGGT-1-s1d1', 'GACTTAGGTTGCGCGA-1-s1d1',
       'GCCTTACTCGTTACAA-1-s1d1', 'GTAAGGTCAATAACCT-1-s1d1',
       ...
       'AGCATCCCACAGAACG-12-s4d9', 'ATTGAAGCACATTGCA-12-s4d9',
       'CTATTCAGTTGTGATG-12-s4d9', 'TGAACAACAGTGAACG-12-s4d9',
       'CGATTCCTCCTGGCTT-12-s4d9', 'AAACCGCGTTTGAGGC-12-s4d9',
       'TGACTTAAGTTCCCGT-12-s4d9', 'GCTGTACCACCGTTCC-12-s4d9',
       'ACACTTGCAACTAGAA-12-s4d9', 'CACTTAAAGTCTGGGC-12-s4d9'],
      dtype='object', length=69249)

In [8]:
new_multiome_obs_names = []
for name in rna_multiome.obs_names:
    parts = name.split('-')
    barcode = parts[0]
    batch = parts[-1]
    name = batch + "_multiome:" + parts[0]
    new_multiome_obs_names.append(name)

In [9]:
rna_multiome.obs_names = new_multiome_obs_names
atac_hvf.obs_names = new_multiome_obs_names

In [10]:
rna_multiome = rna_multiome[multiome_celltypes.index].copy()
rna_multiome.obs = rna_multiome.obs.join(multiome_celltypes)
rna_multiome

AnnData object with n_obs × n_vars = 69247 × 13431
    obs: 'GEX_pct_counts_mt', 'GEX_n_counts', 'GEX_n_genes', 'GEX_size_factors', 'GEX_phase', 'ATAC_nCount_peaks', 'ATAC_atac_fragments', 'ATAC_reads_in_peaks_frac', 'ATAC_blacklist_fraction', 'ATAC_nucleosome_signal', 'cell_type', 'batch', 'ATAC_pseudotime_order', 'GEX_pseudotime_order', 'Samplename', 'Site', 'DonorNumber', 'Modality', 'VendorLot', 'DonorID', 'DonorAge', 'DonorBMI', 'DonorBloodType', 'DonorRace', 'Ethnicity', 'DonorGender', 'QCMeds', 'DonorSmoker', 'l1_cell_type', 'l2_cell_type'
    var: 'feature_types', 'gene_id', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'highly_variable_nbatches', 'highly_variable_intersection'
    uns: 'ATAC_gene_activity_var_names', 'dataset_id', 'genome', 'hvg', 'log1p', 'organism'
    obsm: 'ATAC_gene_activity', 'ATAC_lsi_full', 'ATAC_lsi_red', 'ATAC_umap', 'GEX_X_pca', 'GEX_X_umap'
    layers: 'counts'

### CITE

In [11]:
cite_celltypes = pd.read_csv('data/trimodal/cite_harmonized_celltypes.csv', index_col=0)
cite_celltypes

,l1_cell_type,l2_cell_type
s1d1_cite:GCATTAGCATAAGCGG,B,Naive CD20+ B
s1d1_cite:TACAGGTGTTAGAGTA,Myeloid,CD14+ Mono
s1d1_cite:AGGATCTAGGTCTACT,B,Naive CD20+ B
s1d1_cite:GTAGAAAGTGACACAG,HSC,HSC
s1d1_cite:TCCGAAAAGGATCATA,Erythroid,Reticulocyte
...,...,...
s4d9_cite:GAATCACCACGGAAGT,Other Lympoid,Early Lymphoid
s4d9_cite:GCTGGGTGTACGGATG,T,CD8+ T naive
s4d9_cite:TCGAAGTGTGACAGGT,T,Other T
s4d9_cite:GCAGGCTGTTGCATAC,T,CD4+ T naive


In [12]:
rna_cite.obs_names

Index(['GCATTAGCATAAGCGG-1-s1d1', 'TACAGGTGTTAGAGTA-1-s1d1',
       'AGGATCTAGGTCTACT-1-s1d1', 'GTAGAAAGTGACACAG-1-s1d1',
       'TCCGAAAAGGATCATA-1-s1d1', 'CTCCCAATCCATTGGA-1-s1d1',
       'GACCAATCAATTTCGG-1-s1d1', 'TTCCGGTAGTTGTAAG-1-s1d1',
       'ACCTGTCAGGACTGGT-1-s1d1', 'TTCGATTTCAGGACAG-1-s1d1',
       ...
       'TCTTCCTAGCCAACCC-1-s4d9', 'TTCCACGGTTGAGGAC-1-s4d9',
       'ATTCCTAGTCCAAGAG-1-s4d9', 'GCGGAAAGTACGCGTC-1-s4d9',
       'TAACTTCAGATACAGT-1-s4d9', 'GAATCACCACGGAAGT-1-s4d9',
       'GCTGGGTGTACGGATG-1-s4d9', 'TCGAAGTGTGACAGGT-1-s4d9',
       'GCAGGCTGTTGCATAC-1-s4d9', 'ACGTAACAGGTCTACT-1-s4d9'],
      dtype='object', length=90261)

In [13]:
new_cite_obs_names = []
for name in rna_cite.obs_names:
    parts = name.split('-')
    barcode = parts[0]
    batch = parts[-1]
    name = batch + "_cite:" + parts[0]
    new_cite_obs_names.append(name)

In [14]:
rna_cite.obs_names = new_cite_obs_names
adt.obs_names = new_cite_obs_names

In [15]:
rna_cite = rna_cite[cite_celltypes.index].copy()
rna_cite.obs = rna_cite.obs.join(cite_celltypes)
rna_cite

AnnData object with n_obs × n_vars = 90261 × 13953
    obs: 'GEX_n_genes_by_counts', 'GEX_pct_counts_mt', 'GEX_size_factors', 'GEX_phase', 'ADT_n_antibodies_by_counts', 'ADT_total_counts', 'ADT_iso_count', 'cell_type', 'batch', 'ADT_pseudotime_order', 'GEX_pseudotime_order', 'Samplename', 'Site', 'DonorNumber', 'Modality', 'VendorLot', 'DonorID', 'DonorAge', 'DonorBMI', 'DonorBloodType', 'DonorRace', 'Ethnicity', 'DonorGender', 'QCMeds', 'DonorSmoker', 'is_train', 'l1_cell_type', 'l2_cell_type'
    var: 'feature_types', 'gene_id', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'highly_variable_nbatches', 'highly_variable_intersection'
    uns: 'dataset_id', 'genome', 'hvg', 'log1p', 'organism'
    obsm: 'ADT_X_pca', 'ADT_X_umap', 'ADT_isotype_controls', 'GEX_X_pca', 'GEX_X_umap'
    layers: 'counts'

## Subset genes to highly variable

In [16]:
rna = ad.concat([rna_multiome, rna_cite])
rna

AnnData object with n_obs × n_vars = 159508 × 12059
    obs: 'GEX_pct_counts_mt', 'GEX_size_factors', 'GEX_phase', 'cell_type', 'batch', 'GEX_pseudotime_order', 'Samplename', 'Site', 'DonorNumber', 'Modality', 'VendorLot', 'DonorID', 'DonorAge', 'DonorBMI', 'DonorBloodType', 'DonorRace', 'Ethnicity', 'DonorGender', 'QCMeds', 'DonorSmoker', 'l1_cell_type', 'l2_cell_type'
    obsm: 'GEX_X_pca', 'GEX_X_umap'
    layers: 'counts'

In [17]:
rna.X = rna.layers['counts'].copy()
sc.pp.normalize_total(rna, target_sum=1e4)
sc.pp.log1p(rna)

In [18]:
sc.pp.highly_variable_genes(rna, n_top_genes=4000, batch_key='Samplename')
rna

AnnData object with n_obs × n_vars = 159508 × 12059
    obs: 'GEX_pct_counts_mt', 'GEX_size_factors', 'GEX_phase', 'cell_type', 'batch', 'GEX_pseudotime_order', 'Samplename', 'Site', 'DonorNumber', 'Modality', 'VendorLot', 'DonorID', 'DonorAge', 'DonorBMI', 'DonorBloodType', 'DonorRace', 'Ethnicity', 'DonorGender', 'QCMeds', 'DonorSmoker', 'l1_cell_type', 'l2_cell_type'
    var: 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'highly_variable_nbatches', 'highly_variable_intersection'
    uns: 'log1p', 'hvg'
    obsm: 'GEX_X_pca', 'GEX_X_umap'
    layers: 'counts'

In [19]:
rna_multiome = rna[rna.obs['Modality'] == 'multiome'].copy()
rna_cite = rna[rna.obs['Modality'] == 'cite'].copy()

In [20]:
atac_hvf = atac_hvf[rna_multiome.obs_names].copy()
atac_hvf.obs = atac_hvf.obs.join(rna_multiome.obs[['l1_cell_type', 'l2_cell_type']])

In [21]:
adt = adt[rna_cite.obs_names].copy()
adt.obs = adt.obs.join(rna_cite.obs[['l1_cell_type', 'l2_cell_type']])

## Save files

In [22]:
rna_multiome.write('data/trimodal/rna_multiome_hvg.h5ad')

In [23]:
rna_cite.write('data/trimodal/rna_cite_hvg.h5ad')

In [24]:
atac_hvf.write('data/trimodal/atac_hvf.h5ad')

In [25]:
adt.write('data/trimodal/atac_hvf.h5ad')